# 1. Install and Import dependencies

To install:

`pip install 'git+https://github.com/DLR-RM/stable-baselines3#egg=stable-baselines3[extra]'`

The version on PyPI is currently broken, installing directly from GitHub fixes this.

You may also want to install PyTorch for CUDA, if available:

[PyTorch Install Guide](https://pytorch.org/get-started/locally/)

In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

[CartPole](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)

In [2]:
env_name = 'CartPole-v0'
env = gym.make(env_name)

## Environment Example

In [13]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode {episode+1} finished with score {score}')
env.close()

Episode 1 finished with score 20.0
Episode 2 finished with score 15.0
Episode 3 finished with score 11.0
Episode 4 finished with score 14.0
Episode 5 finished with score 11.0


## Environment Spaces

[Wiki](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html)

![RL Algorithm Comparison](./RL_Alg_Comparison.png)

In [14]:
print(env.observation_space)
print(env.action_space)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Discrete(2)


# 3. Training

In [15]:
log_path = os.path.join('Training', 'Logs')

In [16]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


At this point I got a big long error due to having CUDA 11.4 and PyTorch for CUDA 11.3. I downgraded to CUDA 11.3 using [this guide](https://denishartl.com/installing-cuda-11-3-cudnn-tensorflow-2-4-jupyter-on-a-headless-ubuntu-20-04-server/)

In [17]:
model.learn(total_timesteps=100e3)

Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 1271 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 940          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0074686827 |
|    clip_fraction        | 0.0606       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.688       |
|    explained_variance   | -0.00524     |
|    learning_rate        | 0.0003       |
|    loss                 | 7.17         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0081      |
|    value_loss           | 52.6         |
----------------------------

# 4. Save and Reload Model

In [3]:
PPO_Path = os.path.join('Training', 'Saved_Models', 'PPO_Model_Cartpole')

In [ ]:
model.save(PPO_Path)

In [4]:
del model

NameError: name 'model' is not defined

In [4]:
model = PPO.load(PPO_Path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# 5. Evaluation

In [20]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [21]:
env.close()

# 6. Test

In [6]:
episodes = 5
for episode in range(episodes):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print(f'Episode {episode+1} finished with score {score}')
env.close()

Episode 1 finished with score 200.0
Episode 2 finished with score 200.0
Episode 3 finished with score 200.0
Episode 4 finished with score 200.0
Episode 5 finished with score 200.0


In [10]:
# Render for nice pretty video
from time import time
obs = env.reset()

t_start = time()
while time() - t_start < 60:
    env.render()
    action, _ = model.predict(obs)
    obs, _, _, _ = env.step(action)
env.close()

# 7. Viewing Logs in Tensorboard
Run this in a shell so you don't lock up your notebook.

If Tensorflow not installed, follow [this](https://www.tensorflow.org/install/)

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_2')
!tensorboard --logdir={training_log_path}

2022-04-23 18:12:21.298294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 18:12:21.302389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 18:12:21.302757: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6007/ (Press CTRL

# 8. Adding a Callback to the Training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
save_path = os.path.join('Training', 'Saved_Models')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Changing Policies

In [ ]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 10. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(20000, callback=eval_callback)